In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

evalTime = 11.

In [2]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst

def getTimeLine(path2file, evalTime):
    time = pd.read_csv(pathresults + path2file + "time.txt", names = ["time","Qlight"])["time"][1:] # because we have twice the initial value
    time_diff = abs(np.array(time)-evalTime)
    timeLineNeg = len(time_diff) - np.where(time_diff == min(time_diff))[0][0] 
    timeLinePos = len(time_diff) -timeLineNeg
    print('timeLine',timeLineNeg,timeLinePos, min(time_diff) )
    return timeLineNeg, timeLinePos
    
def get_last_line_as_numpy_array(file_path, timeLine,pos = False, dtype = float, verbose = False):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        
        try:
            if pos:
                last_line = lines[timeLine-1].strip()
            else:
                last_line = lines[-timeLine-1].strip()
        except:
            print('timeLine',file_path, len(lines)-timeLine, len(lines))
            raise Exception

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        if verbose:
            print(file_path, len(array_from_last_line), end=", ")

        return array_from_last_line

def getCylIdx(path2file, timeLinePos):
    fileName = "rhizoSegsId.txt"
    fullpath = path2file+fileName
    rhizoSegsId = get_last_line_as_numpy_array(fullpath, timeLinePos,pos=True, dtype = int, verbose = True)
    print('rhizoSegsId',max(rhizoSegsId))
    return rhizoSegsId
    
def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names

# cylinder max konz (per cell)
def getData_(cid,gId,path2file, timeLineNeg):
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg,pos= False, dtype = float)
    if cid == volIdx:
        return sum(cVol)
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg,pos= False,  dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return sum(cVol)
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, timeLineNeg, pos= False, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    raise Exception

nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16

nToGet = relLenId+1 #elements + 3d coordinates of y , node+vol+len+st 
# baseline_1080_19_10to25_20mn_0s_128
path2file = r'newMucil4p/{}_1476_{}_10to25_20mn_0s_128/'
scenarios = ["earlyDry", "baseline", "lateDry"]
setIds = [17,38,44,85]
exceptPset = []#[('baseline','19'),('baseline','47'),('baseline','83')]
#result_list_compExcept = [path2file.format(scenario, str(setId)) for scenario, setId in exceptPsets]
result_list_comp = [path2file.format(scenario, str(setId)) for scenario in scenarios for setId in setIds if (scenario, str(setId)) not in exceptPset ]

numPset = len(result_list_comp)
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    
    for pSet in allPSets:
        print(pSet,end =", ")
        if True:

        #try:
            #path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
            path2file =result_list_comp[pSet]# "forEGU"+scenarios[pSet]+"_1440_76_10to25_20mn_0s_128/"

            time = pd.read_csv(pathresults + path2file + "time.txt", names = ["time","Qlight"])["time"][1:] # because we have twice the initial value
            timemax = int((max(time))*10)/10
            print('timemax',timemax, timemax==25)
            
            if(timemax==25):
                timeLineNeg, timeLinePos = getTimeLine(path2file, evalTime)
                rr = getCylIdx(path2file, timeLinePos)

                rr.sort()
                print('(',len(rr),')',end =", ")

                for cid in range(nKonz):
                    Ginits = np.array([getData_( cid, gId,path2file, timeLineNeg) for gId in rr])          
                    GiniAll[cid][pSet] = Ginits
                for cid in range(nKonz,volIdx):
                    nodes_ = get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", timeLinePos,pos = True, dtype =float)
                    GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
                #cid = nToGet -1
                Ginits = np.array([getData_( volIdx, gId,path2file, timeLineNeg) for gId in rr])
                GiniAll[volIdx][pSet] = Ginits
                lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", timeLinePos,pos = True, dtype =float, verbose = True)
                GiniAll[lenIdx][pSet] = lens[rr]
                sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", timeLinePos,pos = True, dtype =float, verbose = True)
                assert len(sts) == len(lens)
                GiniAll[stIdx][pSet] = sts[rr]

                orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", timeLinePos,pos = True, dtype =int, verbose = True)
                globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", timeLinePos,pos = True, dtype =int, verbose = True)
                parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", timeLinePos, pos = True,dtype =int, verbose = True)
                ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", timeLinePos,pos = True, dtype =int, verbose = True)
                id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", timeLinePos, pos = True,dtype =int, verbose = True)
                id_roots = id_orgs[np.where(ot_orgs==2)]

                org2basenode = dict(zip(id_orgs, parentNidx))
                #org2ot = dict(zip(id_orgs, ot_orgs))
                isRoot = np.array([oo in id_roots for oo in orgidPerNode])

                orgidPerRoot = orgidPerNode[np.where(isRoot)]
                globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
                segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
                orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
                #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))

                relLens = np.zeros(len(segIdxRoot))
                #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
                for myorg in orgIdxRoot:
                    thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
                    lenSegs = GiniAll[lenIdx][pSet]
                    relLens[thensegs] = np.cumsum(lenSegs[thensegs])


                GiniAll[orgIdId][pSet] = orgIdxRoot
                GiniAll[relLenId][pSet] = relLens
            #except:
            #    print('jump',end =", ")

    return GiniAll #cid pSet rr
    

In [3]:

GiniAll = getData()

0, timemax 25.0 True
timeLine 1009 71 2.842170943040401e-14
newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/rhizoSegsId.txt 194, rhizoSegsId 225
( 194 ), newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/length_Segs.txt 226, newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/subTypes.txt 226, newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/orgidPerNode.txt 226, newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/globalNodeId.txt 226, newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/parentNidx.txt 62, newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/ot_orgs.txt 62, newMucil4p/earlyDry_1476_17_10to25_20mn_0s_128/id_orgs.txt 62, 1, timemax 18.3 False
2, timemax 25.0 True
timeLine 1009 71 2.842170943040401e-14
newMucil4p/earlyDry_1476_44_10to25_20mn_0s_128/rhizoSegsId.txt 196, rhizoSegsId 227
( 196 ), newMucil4p/earlyDry_1476_44_10to25_20mn_0s_128/length_Segs.txt 228, newMucil4p/earlyDry_1476_44_10to25_20mn_0s_128/subTypes.txt 228, newMucil4p/earlyDry_1476_44_10to25_20mn_0s_128/orgidPerNode.txt 228, newMuci

In [4]:
dataInput =  [(scenario, str(setId)) for scenario in scenarios for setId in setIds if (scenario, str(setId)) not in exceptPset ]

In [5]:
# get param set
outputvalsname =  ['wat','cs','cl','coa','cod','cca','ccd','css2','co2','yX','yY','yZ','vol','lenSeg','st','orgId','relLen']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + ['scenario']
df = pd.DataFrame(columns=column_names)

for pset, din in enumerate(dataInput):#range(numPset):
    try:
        N = GiniAll[0][pset].shape[0]
        #assert N == 232
        rows_df = pd.DataFrame()

        for jj, outVn in enumerate(outputvalsname):
            rows_df[outVn] = GiniAll[jj][pset]
            #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
        rows_df['scenario'] = din[0]
        rows_df['pSet'] = din[1]

        df = pd.concat([df, rows_df], ignore_index=True)
    except:
        print('jump',pset, din)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

jump 1 ('earlyDry', '38')


In [7]:
evalTime

11.0

In [8]:
df['time'] = int(evalTime)

In [9]:
df.to_pickle('./4paramTemp'+str(int(evalTime))+'.pkl')

In [10]:
df

,wat,cs,cl,coa,cod,cca,ccd,css2,co2,yX,...,vol,lenSeg,st,orgId,relLen,scenario,pSet,co,cc,time
0,0.042523,0.000003,0.000839,1.163602e-06,0.000008,1.908099e-07,0.000010,0.000067,0.000013,-0.298434,...,0.200000,1.0,0.0,1,1.0,earlyDry,17,0.000009,0.000010,11
1,0.007108,0.000004,0.000837,2.015145e-06,0.000007,2.069498e-07,0.000010,0.000081,0.000014,0.043922,...,0.033333,1.0,1.0,1,2.0,earlyDry,17,0.000009,0.000010,11
2,0.012220,0.000003,0.000838,1.435666e-06,0.000008,1.970010e-07,0.000010,0.000074,0.000014,-0.447197,...,0.057143,1.0,1.0,1,3.0,earlyDry,17,0.000009,0.000010,11
3,0.010999,0.000003,0.000838,1.549024e-06,0.000008,1.987994e-07,0.000010,0.000075,0.000014,-0.104340,...,0.051282,1.0,1.0,1,4.0,earlyDry,17,0.000009,0.000010,11
4,0.011032,0.000003,0.000838,1.535340e-06,0.000008,1.984722e-07,0.000010,0.000075,0.000014,-0.125120,...,0.051282,1.0,1.0,1,5.0,earlyDry,17,0.000009,0.000010,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151,0.218450,0.000007,0.000749,8.904080e-07,0.000008,6.951947e-07,0.000009,0.000163,0.000001,0.941138,...,1.000000,1.0,1.0,4,6.0,lateDry,85,0.000009,0.000009,11
2152,0.110273,0.000007,0.000749,8.926621e-07,0.000008,7.067319e-07,0.000009,0.000163,0.000001,-0.703023,...,0.500000,1.0,1.0,4,7.0,lateDry,85,0.000009,0.000009,11
2153,0.015282,0.000008,0.000749,9.140686e-07,0.000008,7.945960e-07,0.000009,0.000170,0.000001,0.042671,...,0.070175,1.0,1.0,4,8.0,lateDry,85,0.000009,0.000009,11
2154,0.015282,0.000008,0.000749,9.140686e-07,0.000008,7.945960e-07,0.000009,0.000170,0.000001,0.142705,...,0.070175,1.0,1.0,5,1.0,lateDry,85,0.000009,0.000009,11
